<a href="https://colab.research.google.com/github/adammoss/bnn_hmc/blob/main/Mirabest_MCD.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
!pip install astro-datasets --upgrade

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


In [ ]:
!pip install tensorflow_datasets --upgrade

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


In [ ]:
import tensorflow as tf
import numpy as np
import os
import astro_datasets
import tensorflow_datasets as tfds
from matplotlib import pyplot as plt

Load dataset, splitting into training, validation and testing

In [ ]:
dataset_train, info_train = tfds.load(name='mirabest/confident', split='train[:80%]', batch_size=50, as_supervised=True, with_info=True)
dataset_valid, info_valid = tfds.load(name='mirabest/confident', split='train[80%:]', batch_size=50, as_supervised=True, with_info=True) 
dataset_test, info_test = tfds.load(name='mirabest/confident', split='test', batch_size=50, as_supervised=True, with_info=True) 

In [ ]:
#dataset_train, info_train = tfds.load(name='mirabest_confident', split='train', batch_size=729, as_supervised=True, with_info=True)


#for (x,y) in dataset_train:
  #mean = np.mean(x)
  #std = np.std(x)
#print('Mean = ',mean)
#print('Standard deviation = ',std)


In [ ]:
#dataset, info = tfds.load(name='mirabest', split='train', batch_size=1099,as_supervised=True, with_info=True)
#for (x,y) in dataset:
  #mean = np.mean(x)
  #std = np.std(x)
#print('Mean = ',mean)
#print('Standard deviation = ',std)

Define MCD layer

In [ ]:
class MonteCarloDropout(tf.keras.layers.Dropout):
     def call(self, inputs):
         return super().call(inputs, training=True)

Create model

In [ ]:
model = tf.keras.models.Sequential([
    tf.keras.layers.Conv2D(6, kernel_size=5, strides=1, padding = 'same', input_shape=(150, 150, 1)),
    tf.keras.layers.ReLU(),
    tf.keras.layers.MaxPool2D(pool_size=(2, 2)),
    MonteCarloDropout(0.1),
    tf.keras.layers.Conv2D(16, kernel_size=5, strides=1, padding = 'same'),
    tf.keras.layers.ReLU(),
    tf.keras.layers.MaxPool2D(pool_size=(2, 2)),
    MonteCarloDropout(0.1),
    tf.keras.layers.Conv2D(26, kernel_size=5, strides=1, padding = 'same'),
    tf.keras.layers.ReLU(),
    tf.keras.layers.MaxPool2D(pool_size=(2, 2)),
    MonteCarloDropout(0.1),
    tf.keras.layers.Conv2D(32, kernel_size=5, strides=1, padding = 'same'),
    tf.keras.layers.ReLU(),
    tf.keras.layers.MaxPool2D(pool_size=(2, 2)),
    tf.keras.layers.Flatten(),
    MonteCarloDropout(0.1),
    tf.keras.layers.Dense(120),
    tf.keras.layers.ReLU(),
    MonteCarloDropout(0.1),
    tf.keras.layers.Dense(84),
    tf.keras.layers.ReLU(),
    MonteCarloDropout(0.1),
    tf.keras.layers.Dense(2, activation='softmax'),
    ])

In [ ]:
#model.summary()

Loss function

In [ ]:
loss=tf.keras.losses.SparseCategoricalCrossentropy()

Optimiser and learning rate scheduler

In [ ]:
opt = tf.keras.optimizers.Adam(learning_rate=5e-5)
reduce_lr = tf.keras.callbacks.ReduceLROnPlateau(monitor='val_loss', factor=0.95, patience=3)

Compile model

In [ ]:
model.compile(optimizer=opt,
              loss=loss,
              metrics='accuracy')


Save best weights

In [ ]:
checkpoint_path = "mirabest/cp.ckpt"
checkpoint_dir = os.path.dirname(checkpoint_path)

cp_callback = tf.keras.callbacks.ModelCheckpoint(filepath=checkpoint_path,
                                                  save_weights_only=False,
                                                  monitor='val_loss',
                                                  mode='min',
                                                  verbose=1,
                                                  save_best_only=True)

Train model

In [ ]:


model.fit(dataset_train, epochs=100, callbacks=[reduce_lr, cp_callback], validation_data=dataset_valid)




Epoch 1/100
12/12 [==============================] - ETA: 0s - loss: 3.6766 - accuracy: 0.5026
Epoch 1: val_loss improved from inf to 2.53051, saving model to mirabest/cp.ckpt
INFO:tensorflow:Assets written to: mirabest/cp.ckpt/assets


INFO:tensorflow:Assets written to: mirabest/cp.ckpt/assets


12/12 [==============================] - 6s 229ms/step - loss: 3.6766 - accuracy: 0.5026 - val_loss: 2.5305 - val_accuracy: 0.5068 - lr: 5.0000e-05
Epoch 2/100
 9/12 [=====================>........] - ETA: 0s - loss: 2.3120 - accuracy: 0.5000
Epoch 2: val_loss improved from 2.53051 to 1.32829, saving model to mirabest/cp.ckpt
INFO:tensorflow:Assets written to: mirabest/cp.ckpt/assets


INFO:tensorflow:Assets written to: mirabest/cp.ckpt/assets


12/12 [==============================] - 9s 816ms/step - loss: 2.1908 - accuracy: 0.5026 - val_loss: 1.3283 - val_accuracy: 0.5548 - lr: 5.0000e-05
Epoch 3/100
11/12 [==========================>...] - ETA: 0s - loss: 1.4177 - accuracy: 0.5545
Epoch 3: val_loss did not improve from 1.32829
12/12 [==============================] - 0s 32ms/step - loss: 1.3889 - accuracy: 0.5557 - val_loss: 1.4026 - val_accuracy: 0.4932 - lr: 5.0000e-05
Epoch 4/100
 9/12 [=====================>........] - ETA: 0s - loss: 1.1721 - accuracy: 0.5600
Epoch 4: val_loss improved from 1.32829 to 1.12379, saving model to mirabest/cp.ckpt
INFO:tensorflow:Assets written to: mirabest/cp.ckpt/assets


...

12/12 [==============================] - 5s 473ms/step - loss: 0.3966 - accuracy: 0.8268 - val_loss: 0.5333 - val_accuracy: 0.7671 - lr: 2.0906e-05
Epoch 77/100
10/12 [========================>.....] - ETA: 0s - loss: 0.3944 - accuracy: 0.8340
Epoch 77: val_loss did not improve from 0.53332
12/12 [==============================] - 0s 27ms/step - loss: 0.4003 - accuracy: 0.8319 - val_loss: 0.6159 - val_accuracy: 0.6849 - lr: 2.0906e-05
Epoch 78/100
12/12 [==============================] - ETA: 0s - loss: 0.4161 - accuracy: 0.7890
Epoch 78: val_loss improved from 0.53332 to 0.49197, saving model to mirabest/cp.ckpt
INFO:tensorflow:Assets written to: mirabest/cp.ckpt/assets


INFO:tensorflow:Assets written to: mirabest/cp.ckpt/assets


12/12 [==============================] - 7s 599ms/step - loss: 0.4161 - accuracy: 0.7890 - val_loss: 0.4920 - val_accuracy: 0.7945 - lr: 2.0906e-05
Epoch 79/100
10/12 [========================>.....] - ETA: 0s - loss: 0.4177 - accuracy: 0.8220
Epoch 79: val_loss did not improve from 0.49197
12/12 [==============================] - 0s 29ms/step - loss: 0.4081 - accuracy: 0.8285 - val_loss: 0.6204 - val_accuracy: 0.6986 - lr: 2.0906e-05
Epoch 80/100
11/12 [==========================>...] - ETA: 0s - loss: 0.4147 - accuracy: 0.8073
Epoch 80: val_loss did not improve from 0.49197
12/12 [==============================] - 0s 28ms/step - loss: 0.4166 - accuracy: 0.8062 - val_loss: 0.5958 - val_accuracy: 0.6918 - lr: 2.0906e-05
Epoch 81/100
 9/12 [=====================>........] - ETA: 0s - loss: 0.3953 - accuracy: 0.8267
Epoch 81: val_loss did not improve from 0.49197
12/12 [==============================] - 0s 20ms/step - loss: 0.4007 - accuracy: 0.8199 - val_loss: 0.5223 - val_accuracy: 0.7

INFO:tensorflow:Assets written to: mirabest/cp.ckpt/assets


12/12 [==============================] - 2s 167ms/step - loss: 0.3944 - accuracy: 0.8250 - val_loss: 0.4235 - val_accuracy: 0.7808 - lr: 1.6177e-05
Epoch 96/100
11/12 [==========================>...] - ETA: 0s - loss: 0.3960 - accuracy: 0.8182
Epoch 96: val_loss did not improve from 0.42348
12/12 [==============================] - 0s 22ms/step - loss: 0.3947 - accuracy: 0.8199 - val_loss: 0.5574 - val_accuracy: 0.7260 - lr: 1.6177e-05
Epoch 97/100
 9/12 [=====================>........] - ETA: 0s - loss: 0.4297 - accuracy: 0.7844
Epoch 97: val_loss did not improve from 0.42348
12/12 [==============================] - 0s 20ms/step - loss: 0.4089 - accuracy: 0.8010 - val_loss: 0.5302 - val_accuracy: 0.6986 - lr: 1.6177e-05
Epoch 98/100
 9/12 [=====================>........] - ETA: 0s - loss: 0.4158 - accuracy: 0.8222
Epoch 98: val_loss did not improve from 0.42348
12/12 [==============================] - 0s 20ms/step - loss: 0.3978 - accuracy: 0.8233 - val_loss: 0.6050 - val_accuracy: 0.6

Test model

In [ ]:

new_model = tf.keras.models.load_model('mirabest/cp.ckpt')

#label_predict = new_model.predict(dataset_test)


In [ ]:
for _ in range(10):
  correct, n = 0, 0
  for (x, y) in dataset_test:
    yhat = new_model.predict(x)
    correct += np.sum(np.argmax(yhat, axis=-1) == y)
    n += len(y)
  print('Accuracy:',correct / n)

Accuracy: 0.7307692307692307
Accuracy: 0.6923076923076923
Accuracy: 0.7692307692307693
Accuracy: 0.7307692307692307
Accuracy: 0.7596153846153846
Accuracy: 0.7692307692307693
Accuracy: 0.7596153846153846
Accuracy: 0.7788461538461539
Accuracy: 0.7596153846153846
Accuracy: 0.7596153846153846


Predictive entropy, mutual information, average entropy

In [ ]:

samples_iter = 200
softmax = []

for _ in range(samples_iter):
  s = []
  for (x, y) in dataset_test:
    pred = new_model.predict(x)
    #softmax.append(pred)
    s.append(pred)
  s = np.concatenate(s)
  softmax.append(s)

softmax = np.array(softmax)



def entropy_MI(softmax, samples_iter):

    class_0 = softmax[:][0]
    class_1 = softmax[:][1]
    #biased estimator of predictive entropy, bias will reduce as samples_iter is increased
    predictive_entropy = -((np.sum(class_0)/samples_iter) * np.log(np.sum(class_0)/samples_iter) + (np.sum(class_1)/samples_iter) * np.log(np.sum(class_1)/samples_iter))
    
    #Orignal: -((np.sum(class_0)/samples_iter) * np.log(np.sum(class_0)/samples_iter) + (np.sum(class_1)/samples_iter) * np.log(np.sum(class_1)/samples_iter))
    

    mutual_info = predictive_entropy + np.sum((class_0 * np.log(class_0)) + (class_1 * np.log(class_1)))/samples_iter

    #Original: predictive_entropy + np.sum(class_0*np.log(class_0))/samples_iter +  np.sum(class_1*np.log(class_1))/samples_iter
    
    #entropy of a single pass
    class0 = softmax[:][0][0]
    class1 = softmax[:][1][0]
    entropy_singlepass = -np.sum((class0 * np.log(class0)) + (class1 * np.log(class1)))/samples_iter

    #Original: -(class0 * np.log(class0) + class1 * np.log(class1))

    #print("Entropy of a single pass:", entropy_singlepass)
    return predictive_entropy, mutual_info, entropy_singlepass



predictive_entropy, mutual_info, entropy_singlepass = entropy_MI(softmax, samples_iter)


print("Predictive Entropy:", predictive_entropy)
print("Mutual Information:", mutual_info)
print("Entropy of a single pass:", entropy_singlepass)

Predictive Entropy: 0.6800835261029305
Mutual Information: 0.24403707834902427
Entropy of a single pass: 0.0028205105662345888
